In [3]:
import pandas as pd

df = pd.read_csv('data/others/어린이집.csv')
df.head(), df.shape, df['지역별'].unique()


(     시점    지역별  직장어린이집  어린이집
 0  2010  서울특별시     109  5870
 1  2010  부산광역시      19  1689
 2  2010  대구광역시      14  1544
 3  2010  인천광역시      26  1857
 4  2010  광주광역시      15  1192,
 (236, 4),
 array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
        '경기도', '강원특별자치도', '충청북도', '충청남도', '전북특별자치도', '전라남도', '경상북도',
        '경상남도', '제주특별자치도', '세종특별자치시'], dtype=object))

In [4]:
df['지역별'] = df['지역별'].replace({'서울특별시': '서울', '부산광역시': '부산', '대구광역시': '대구',
                               '인천광역시': '인천', '광주광역시': '광주', '대전광역시': '대전',
                               '울산광역시': '울산', '경기도': '경기', '강원특별자치도': '강원',
                               '충청북도': '충북', '충청남도': '충남', '전북특별자치도': '전북',
                               '전라남도': '전남', '경상북도': '경북', '경상남도': '경남', '제주특별자치도': '제주'})

In [5]:
# '시점'을 datetime 형식으로 변환
df['시점'] = pd.to_datetime(df['시점'].astype(str) + '-01-01')

# 인덱스로 설정
yearly_data = df.set_index('시점')

# 지역별로 그룹화 후 리샘플링
monthly_dfs = []

for region, group in yearly_data.groupby('지역별'):
    monthly_df = group.resample('ME').agg({'어린이집': 'mean'}).round(3)
    
    # 보간
    monthly_df = monthly_df.interpolate(method='linear')
    
    # NaN 값 처리
    monthly_df.fillna(0, inplace=True)
    
    # 시점 다시 설정
    monthly_df.reset_index(inplace=True)
    
    # '시점'이 datetime인지 확인 후, 형식 변경
    if pd.api.types.is_datetime64_any_dtype(monthly_df['시점']):
        monthly_df['시점'] = monthly_df['시점'].dt.strftime('%Y.%m')
    
    # 지역별 컬럼 추가
    monthly_df['지역별'] = region
    
    monthly_dfs.append(monthly_df)

# 모든 지역별 데이터를 하나의 DataFrame으로 결합
final_monthly_df = pd.concat(monthly_dfs, ignore_index=True)
final_monthly_df['어린이집'] = final_monthly_df['어린이집'].map(lambda x: f"{x:.3f}")

# 결과 확인
print(final_monthly_df.tail(12), final_monthly_df.shape, final_monthly_df['지역별'].unique())

# 데이터 저장
final_monthly_df.to_csv('data/others/어린이집(월별).csv', index=False)


           시점     어린이집 지역별
2633  2022.02  967.750  충북
2634  2022.03  963.500  충북
2635  2022.04  959.250  충북
2636  2022.05  955.000  충북
2637  2022.06  950.750  충북
2638  2022.07  946.500  충북
2639  2022.08  942.250  충북
2640  2022.09  938.000  충북
2641  2022.10  933.750  충북
2642  2022.11  929.500  충북
2643  2022.12  925.250  충북
2644  2023.01  921.000  충북 (2645, 3) ['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종특별자치시' '울산' '인천' '전남'
 '전북' '제주' '충남' '충북']
